In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role, image_uris
from sagemaker.sklearn.model import SKLearnModel

# Assume that you've already saved your model to a file named 'model.pkl'
s3_model_path = 's3://namthai-bigdata-bucket/model/pipeline.tar.gz'


# retrive image uri
image_uri = image_uris.retrieve(framework='sklearn', region='ap-southeast-1',
                                version='1.2-1',
                                image_scope='inference')

# endpoint name
endpoint_name = 'v-twac-2024'


# Get the execution role for SageMaker
role = get_execution_role()



# Create a SageMaker Scikit-learn Model
sklearn_model = SKLearnModel(model_data=s3_model_path,
                             role=role,
                             image_uri=image_uri,
                             entry_point='inference.py',
                             dependencies=['requirements.txt', 'transformer.py'])
                             
                             

# Deploy the model to a real-time endpoint
predictor = sklearn_model.deploy(initial_instance_count=1,
                         instance_type='ml.m5.xlarge',
                         endpoint_name=endpoint_name,
                         logs=True)

In [ ]:
# Remove the endpoint
predictor.delete_endpoint(delete_endpoint_config=True)

sagemaker = boto3.client('sagemaker')

sagemaker.delete_endpoint(EndpointName='v-twac-2024')